<a href="https://colab.research.google.com/github/Elwing-Chou/ml1216/blob/main/news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
# pd.options.display.max_rows = 1000
# pd.options.display.max_colwidth = 1000

In [ ]:
import zipfile
f = zipfile.ZipFile("drive/MyDrive/news/chinese_news_trans.zip")
f.extractall()
f = zipfile.ZipFile("drive/MyDrive/news/chinese_news_test.zip")
f.extractall()

In [ ]:
import os
import glob
import pandas as pd
def load_data(base):
    datas = {
        "content":[],
        "category":[]
    }
    for dn in glob.glob("{}/*".format(base)):
        small = os.path.join(dn, "*.txt")
        big = os.path.join(dn, "*.TXT")
        for fn in glob.glob(small) + glob.glob(big):
            with open(fn, encoding="utf-8") as f:
                news = f.read()
                datas["content"].append(news)
                cat = os.path.split(dn)[-1]
                datas["category"].append(cat)
    return pd.DataFrame(datas)
train = load_data("chinese_news_trans")
test = load_data("chinese_news_test")

In [ ]:
test

In [ ]:
# Series: unique/value_counts
writers = train["category"].unique()
trans = {n:i for i, n in enumerate(writers)}
trans_r = {i:n for i, n in enumerate(writers)}
# Series: replace
y_train = train["category"].replace(trans)
y_test = test["category"].replace(trans)
y_test

In [ ]:
import jieba
from urllib.request import urlretrieve
url = "https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big"
urlretrieve(url, "dict.txt.big")
jieba.set_dictionary("dict.txt.big")

In [ ]:
def newscut(p):
    return " ".join(jieba.cut(p))
x_train = train["content"].apply(newscut)
x_test = test["content"].apply(newscut)
x_test

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
x_train_count = vec.fit_transform(x_train)
x_test_count = vec.transform(x_test)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha=0.01)
clf.fit(x_train_count, y_train)

In [ ]:
from sklearn.metrics import accuracy_score
pre = clf.predict(x_test_count)
accuracy_score(pre, y_test)

In [ ]:
p = input("寫首新聞:")
p = [newscut(p)]
count = vec.transform(p)
prob = clf.predict_proba(count)[0]
for w, pr in zip(writers, prob):
    print(w, round(pr, 4))
ans = clf.predict(count)[0]
print("最可能是:", writers[ans])